In [3]:
import json
import pandas as pd
from sqlanalyzer import column_parser, query_analyzer

IndentationError: expected an indented block (column_parser.py, line 203)

In [ ]:
def extract_subquery_fields(query, db_fields, **kwargs):
    formatter = column_parser.Parser(query)
    formatted = formatter.format_query(query)
    fields = formatter.match_queried_fields(formatted, db_fields, **kwargs)
    return fields


def unnest_query_list(query_list):
    preprocess_list = []
    
    for q in query_list:
        for _, query in q.items():
            
            if isinstance(query, str):
                preprocess_list.append(query)
            else:
                
                for sub_q in query:
                    sub_list = []
                    for _, sub_query in sub_q.items():
                        
                        if isinstance(sub_query, str):
                            sub_list.append(sub_query)
                            
                        else:
                            for sub_sub_q in sub_query:
                                for _, sub_sub_query in sub_sub_q.items():
                                    
                                    if isinstance(sub_sub_query, str):
                                        sub_list.append(sub_sub_query)
                                        
                    preprocess_list.extend(sub_list)
                    
    return preprocess_list
    

In [2]:
# query = open('active_devs.sql').read()
query = """SELECT rrr.*,
       CASE
           WHEN entexc.sfdc_acct_id IS NOT NULL THEN TRUE
           ELSE FALSE
       END AS ent_exception,
       CASE
           WHEN pfree.mbx_acct_id IS NOT NULL THEN TRUE
           ELSE FALSE
       END AS paygo_free,
       meta.sku_type
FROM analytics.rack_rate_revenue rrr
LEFT JOIN enterprise_exception entexc ON rrr.sfdc_acct_id = entexc.sfdc_acct_id
AND rrr.mbx_acct_lvl = 'enterprise'
LEFT JOIN paygo_free pfree ON rrr.mbx_acct_id = pfree.mbx_acct_id
AND pfree.date_month BETWEEN DATE_ADD('month', -1, DATE(vdate)) AND DATE_ADD('day', -1, DATE(vdate))
LEFT JOIN analytics.sku_metadata meta ON rrr.sku_id = meta.sku_id
WHERE rrr.dt = '2020-06-30'
"""
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
cte_queries = formatter.parse_cte(formatted_query)


NameError: name 'column_parser' is not defined

In [ ]:
with open('./data.json', 'r') as f:
    query_dict = json.load(f)

In [ ]:
print(json.dumps(query_dict, indent=2), '\n\n\n')

In [ ]:
query_dict.keys()

In [ ]:
print(cte_queries['main_query'])

## metastore 

In [ ]:
db_fields_1 = pd.DataFrame({'db_table': 'analytics.rack_rate_revenue', 
            'all_columns': ['vdate',
 'mbx_acct_id',
 'mbx_acct_lvl',
 'is_apa',
 'sfdc_acct_id',
 'sfdc_acct_name',
 'sfdc_acct_owner',
 'pod',
 'service_group',
 'sku_id',
 'subunits',
 'mtd_usage',
 'daily_revenue',
 'mtd_revenue',
 'discount_rate',
 'daily_revenue_w_discount',
 'mtd_revenue_w_discount',
 'dt']})


In [ ]:
db_fields_2 = pd.DataFrame({'db_table': 'analytics.sku_metadata', 
            'all_columns': ['sku_id', 'sku_ty', 'sku_na']})


In [ ]:
db_fields = db_fields_1.append(db_fields_2, ignore_index=True)
db_fields

In [ ]:
raw_query = cte_queries['main_query']
formatter = column_parser.Parser(raw_query)
formatted = formatter.format_query(raw_query)
analyzer = query_analyzer.Analyzer(formatted)
query_dict = analyzer.parse_query(formatted)
preprocess_list = unnest_query_list(query_dict)

In [ ]:
preprocess_list

In [ ]:
extract_subquery_fields(preprocess_list[0], db_fields)